#### N.b. this notebook is from the [IceNet notebooks repo](https://github.com/icenet-ai/icenet-notebooks/blob/main/pytorch/1_icenet_forecast_unet.ipynb)

PyTorch UNet implementation using IceNet library for data download and post-processing of sea ice forecasting.

This notebook has been designed to be independent of other notebooks.

### Highlights
The key features of this notebook are:
* [1. Download](#1.-Download) 
* [2. Data Processing](#2.-Data-Processing)
* [3. Train](#3.-Train)
* [4. Prediction](#4.-Prediction)
* [5. Outputs and Plotting](#5.-Outputs-and-Plotting)

Please note that this notebook relies on a pytorch data loader implementation which is only available from icenet v0.2.8+.

To install the necessary python packages, you can use the conda `icenet-notebooks/pytorch/environment.yml` environment file on a Linux system to be able to set-up the necessary pytorch + tensorflow + CUDA + other modules which could be a tricky mix to get working manually:

```bash
conda env create -f environment.yml
```

### Contributions
#### PyTorch implementation of IceNet

Andrew McDonald ([icenet-gan](https://github.com/ampersandmcd/icenet-gan))

Bryn Noel Ubald (Refactor, updates for daily predictions and matching icenet library)

#### Notebook
Bryn Noel Ubald (author)

#### PyTorch Integration
Bryn Noel Ubald

Ryan Chan

In [ ]:
import os
import numpy as np
import pandas as pd
from pathlib import Path

import torch
from torch.utils.data import DataLoader

# We also set the logging level so that we get some feedback from the API
import logging

logging.basicConfig(level=logging.INFO)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.set_float32_matmul_precision("medium")

## 1. Download

In [ ]:
# Set the base path where data should be stored. This should be the same as what you use in your IceStationZebra config.
base_path = "/LOCAL/PATH/WHERE/YOU/WANT/TO/STORE/DATA/"

In [ ]:
data_path = (Path(base_path) / "data" / "notebooks" / "icenet").resolve()
current_directory = Path.cwd()

In [ ]:
import numpy
from icenet.data.sic.mask import Masks
from icenet.data.interfaces.cds import ERA5Downloader
from icenet.data.sic.osisaf import SICDownloader

### Mask data

Create masks for masking data.

In [ ]:
# Unset SLURM_NTASKS if it's causing issues
if "SLURM_NTASKS" in os.environ:
    del os.environ["SLURM_NTASKS"]

# Optionally, set SLURM_NTASKS_PER_NODE if needed
os.environ["SLURM_NTASKS_PER_NODE"] = "1"  # or whatever value is appropriate

In [ ]:
masks = Masks(north=False, south=True)
masks.generate(save_polarhole_masks=False)

### Climate and Sea Ice data

Download climate variables from ERA5 and sea ice concentration from OSI-SAF.

:::note
The prototype data currently in use (South Pole, 2020) can be downloaded from **Baskerville** at the following path: `/vjgo8416-ice-frcst/shared/prototype_data/`
:::

#### Download ERA5 Data  

Use the following link to set up the ERA5 API: [https://cds.climate.copernicus.eu/how-to-api?](https://cds.climate.copernicus.eu/how-to-api?).



In [ ]:
os.chdir(data_path)

era5 = ERA5Downloader(
    var_names=["tas", "zg", "uas", "vas"],  # Name of variables to download
    levels=[
        None,
        [250, 500],
        None,
        None,
    ],  # The levels at which to obtain the variables for (e.g. for zg, it is the pressure levels)
    dates=[
        pd.to_datetime(date).date()
        for date in pd.date_range(  # Dates to download the variable data for
            "2019-12-01", "2021-01-31", freq="D"
        )
    ],
    delete_tempfiles=False,  # Whether to delete temporary downloaded files
    max_threads=64,  # Maximum number of concurrent downloads
    north=False,  # Boolean: Whether require data across northern hemisphere
    south=True,  # Boolean: Whether require data across southern hemisphere
    # NOTE: there appears to be a bug with the toolbox API at present (icenet#54)
    use_toolbox=False,  # Experimental, alternative download method
)

era5.download()  ## Uncomment to download the data

Re-grid ERA5 reanalysis data, and rotate wind vector data from ERA5 to align with EASE2 projection.

In [ ]:
era5.regrid()
era5.rotate_wind_data()
os.chdir(current_directory)

#### DOWNLOAD SIC Data  


In [ ]:
os.chdir(data_path)
sic = SICDownloader(
    dates=[
        pd.to_datetime(date).date()
        for date in pd.date_range(  # Dates to download SIC data for
            "2020-01-01", "2020-04-30", freq="D"
        )
    ],
    delete_tempfiles=False,  # Delete temporary downloaded files after use
    north=False,  # Use mask for the Northern Hemisphere (set to True if needed)
    south=True,  # Use mask for the Southern Hemisphere
    parallel_opens=True,  # Enable parallel processing with dask.delayed
)

sic.download()  ## Uncomment to download the data
os.chdir(current_directory)

## 2. Data Processing

Process downloaded datasets.

To make life easier, setting up train, val, test dates.

In [ ]:
processing_dates = dict(
    train=[pd.to_datetime(el) for el in pd.date_range("2020-01-01", "2020-12-31")],
    val=[pd.to_datetime(el) for el in pd.date_range("2021-01-01", "2021-01-31")],
    test=[pd.to_datetime(el) for el in pd.date_range("2012-12-01", "2019-12-31")],
)
processed_name = "notebook_api_pytorch_data"

Next, we create the data producer and configure them for the dataset we want to create.

In [ ]:
from icenet.data.processors.era5 import IceNetERA5PreProcessor
from icenet.data.processors.meta import IceNetMetaPreProcessor
from icenet.data.processors.osi import IceNetOSIPreProcessor

pp = IceNetERA5PreProcessor(
    ["uas", "vas"],
    ["tas", "zg500", "zg250"],
    processed_name,
    processing_dates["train"],
    processing_dates["val"],
    processing_dates["test"],
    linear_trends=tuple(),
    north=False,
    south=True,
)

osi = IceNetOSIPreProcessor(
    ["siconca"],
    [],
    processed_name,
    processing_dates["train"],
    processing_dates["val"],
    processing_dates["test"],
    linear_trends=tuple(),
    north=False,
    south=True,
)

meta = IceNetMetaPreProcessor(processed_name, north=False, south=True)

Next, we initialise the data processors using `init_source_data` which scans the data source directories to understand what data is available for processing based on the parameters. Since we named the processed data `"notebook_api_data"` above, it will create a data loader config file, `loader.notebook_api_data.json`, in the current directory.

In [ ]:
# Causes hanging on training, when generating sample.
# Sometimes struggles to access files, re-running seems to fix it

pp.init_source_data(
    lag_days=1,
)
pp.process()

osi.init_source_data(
    lag_days=1,
)
osi.process()

meta.process()

At this point the preprocessed data is ready to convert or create a configuration for the network dataset.

### Dataset creation

As with the `icenet_dataset_create` command we can create a dataset configuration for training the network. As before this can include cached data for the network in the format of a TFRecordDataset compatible set of tfrecords. To achieve this we create the `IceNetDataLoader`, which can both generate `IceNetDataSet` configurations (which easily provide the necessary functionality for training and prediction) as well as individual data samples for direct usage.

In [ ]:
from icenet.data.loaders import IceNetDataLoaderFactory

implementation = "dask"
loader_config = "loader.notebook_api_pytorch_data.json"
dataset_name = "notebook_api_pytorch_data"
lag = 1

dl = IceNetDataLoaderFactory().create_data_loader(
    implementation,
    loader_config,
    dataset_name,
    lag,
    n_forecast_days=7,
    north=False,
    south=True,
    output_batch_size=1,
    generate_workers=4,
)

At this point we can either use `generate` or `write_dataset_config_only` to produce a ready-to-go `IceNetDataSet` configuration. Both of these will generate a dataset config, `dataset_config.notebook_api_pytorch_data.json` (recall we set the dataset name as `notebook_api_pytorch_data` above).

In this case, for pytorch, will read data in directly, rather than using cached tfrecords inputs.

In [ ]:
dl.write_dataset_config_only()

We can now create the IceNetDataSet object:

In [ ]:
from icenet.data.dataset import IceNetDataSetPyTorch

dataset_config = f"dataset_config.{dataset_name}.json"

In [ ]:
batch_size = 4
shuffle = False
persistent_workers = True
num_workers = 4

## 3. Train

We implement a custom PyTorch class for training.

## IceNet2 UNet model

PyTorch UNet implementation based on current state of IceNet2, and Andrew's work from https://github.com/ampersandmcd/icenet-gan/.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Interpolate(nn.Module):
    def __init__(self, scale_factor, mode):
        super().__init__()
        self.interp = F.interpolate
        self.scale_factor = scale_factor
        self.mode = mode

    def forward(self, x):
        x = self.interp(x, scale_factor=self.scale_factor, mode=self.mode)
        return x


class UNet(nn.Module):
    def __init__(
        self,
        input_channels,
        filter_size=3,
        n_filters_factor=1,
        n_forecast_days=7,
        n_output_classes=1,
        **kwargs,
    ):
        super(UNet, self).__init__()

        self.input_channels = input_channels
        self.filter_size = filter_size
        self.n_filters_factor = n_filters_factor
        self.n_forecast_days = n_forecast_days
        self.n_output_classes = n_output_classes

        start_out_channels = 64
        reduced_channels = int(start_out_channels * n_filters_factor)
        channels = {
            start_out_channels * 2**pow: reduced_channels * 2**pow for pow in range(4)
        }

        # Encoder
        self.conv1 = self.conv_block(input_channels, channels[64])
        self.conv2 = self.conv_block(channels[64], channels[128])
        self.conv3 = self.conv_block(channels[128], channels[256])
        self.conv4 = self.conv_block(channels[256], channels[256])

        # Bottleneck
        self.conv5 = self.bottleneck_block(channels[256], channels[512])

        # Decoder
        self.up6 = self.upconv_block(channels[512], channels[256])
        self.up7 = self.upconv_block(channels[256], channels[256])
        self.up8 = self.upconv_block(channels[256], channels[128])
        self.up9 = self.upconv_block(channels[128], channels[64])

        self.up6b = self.conv_block(channels[512], channels[256])
        self.up7b = self.conv_block(channels[512], channels[256])
        self.up8b = self.conv_block(channels[256], channels[128])
        self.up9b = self.conv_block(channels[128], channels[64], final=True)

        # Final layer
        self.final_layer = nn.Conv2d(
            channels[64], n_forecast_days, kernel_size=1, padding="same"
        )

    def forward(self, x):
        # transpose from shape (b, h, w, c) to (b, c, h, w) for pytorch conv2d layers
        x = torch.movedim(x, -1, 1)  # move c from last to second dim

        # Encoder
        bn1 = self.conv1(x)
        conv1 = F.max_pool2d(bn1, kernel_size=2)
        bn2 = self.conv2(conv1)
        conv2 = F.max_pool2d(bn2, kernel_size=2)
        bn3 = self.conv3(conv2)
        conv3 = F.max_pool2d(bn3, kernel_size=2)
        bn4 = self.conv4(conv3)
        conv4 = F.max_pool2d(bn4, kernel_size=2)

        # Bottleneck
        bn5 = self.conv5(conv4)

        # Decoder
        up6 = self.up6b(torch.cat([bn4, self.up6(bn5)], dim=1))
        up7 = self.up7b(torch.cat([bn3, self.up7(up6)], dim=1))
        up8 = self.up8b(torch.cat([bn2, self.up8(up7)], dim=1))
        up9 = self.up9b(torch.cat([bn1, self.up9(up8)], dim=1))

        # Final layer
        output = self.final_layer(up9)

        # transpose from shape (b, c, h, w) back to (b, h, w, c) to align with training data
        output = torch.movedim(output, 1, -1)  # move c from second to final dim

        b, h, w, c = output.shape

        # unpack c=classes*months dimension into classes, months as separate dimensions
        output = output.reshape((b, h, w, self.n_output_classes, self.n_forecast_days))

        return output

    def conv_block(self, in_channels, out_channels, final=False):
        block = nn.Sequential(
            nn.Conv2d(
                in_channels, out_channels, kernel_size=self.filter_size, padding="same"
            ),
            nn.ReLU(inplace=True),
            nn.Conv2d(
                out_channels, out_channels, kernel_size=self.filter_size, padding="same"
            ),
            nn.ReLU(inplace=True),
        )
        if not final:
            batch_norm = nn.Sequential(
                nn.BatchNorm2d(num_features=out_channels),
            )
            return nn.Sequential().extend(block).extend(batch_norm)
        else:
            final_block = nn.Sequential(
                nn.Conv2d(
                    out_channels,
                    out_channels,
                    kernel_size=self.filter_size,
                    padding="same",
                ),
                nn.ReLU(inplace=True),
            )
            return nn.Sequential().extend(block).extend(final_block)

    def bottleneck_block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(
                in_channels, out_channels, kernel_size=self.filter_size, padding="same"
            ),
            nn.ReLU(inplace=True),
            nn.Conv2d(
                out_channels, out_channels, kernel_size=self.filter_size, padding="same"
            ),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(num_features=out_channels),
        )

    def upconv_block(self, in_channels, out_channels):
        return nn.Sequential(
            Interpolate(scale_factor=2, mode="nearest"),
            nn.Conv2d(in_channels, out_channels, kernel_size=2, padding="same"),
            nn.ReLU(inplace=True),
        )

Define custom metrics for use in validation and monitoring

In [ ]:
from torchmetrics import Metric


class IceNetAccuracy(Metric):
    """Binary accuracy metric for use at multiple leadtimes.

    Reference: https://lightning.ai/docs/torchmetrics/stable/pages/implement.html
    """

    # Set class properties
    is_differentiable: bool = False
    higher_is_better: bool = True
    full_state_update: bool = True

    def __init__(self, leadtimes_to_evaluate: list):
        """Custom loss/metric for binary accuracy in classifying SIC>15% for multiple leadtimes.

        Args:
            leadtimes_to_evaluate: A list of leadtimes to consider
                e.g., [0, 1, 2, 3, 4, 5] to consider first six days in accuracy computation or
                e.g., [0] to only look at the first day's accuracy
                e.g., [5] to only look at the sixth day's accuracy
        """
        super().__init__()
        self.leadtimes_to_evaluate = leadtimes_to_evaluate
        self.add_state(
            "weighted_score", default=torch.tensor(0.0), dist_reduce_fx="sum"
        )
        self.add_state(
            "possible_score", default=torch.tensor(0.0), dist_reduce_fx="sum"
        )

    def update(
        self, preds: torch.Tensor, target: torch.Tensor, sample_weight: torch.Tensor
    ):
        # preds and target are shape (b, h, w, t)
        preds = (preds > 0.15).long()  # torch.Size([2, 432, 432, 7])
        target = (target > 0.15).long()  # torch.Size([2, 432, 432, 7])
        base_score = (
            preds[:, :, :, self.leadtimes_to_evaluate]
            == target[:, :, :, self.leadtimes_to_evaluate]
        )
        self.weighted_score += torch.sum(
            base_score * sample_weight[:, :, :, self.leadtimes_to_evaluate]
        )
        self.possible_score += torch.sum(
            sample_weight[:, :, :, self.leadtimes_to_evaluate]
        )

    def compute(self):
        return self.weighted_score.float() / self.possible_score * 100.0


class SIEError(Metric):
    """
    Sea Ice Extent error metric (in km^2) for use at multiple leadtimes.
    """

    # Set class properties
    is_differentiable: bool = False
    higher_is_better: bool = False
    full_state_update: bool = True

    def __init__(self, leadtimes_to_evaluate: list):
        """Construct an SIE error metric (in km^2) for use at multiple leadtimes.
        leadtimes_to_evaluate: A list of leadtimes to consider
            e.g., [0, 1, 2, 3, 4, 5] to consider six days in computation or
            e.g., [0] to only look at the first day
            e.g., [5] to only look at the sixth day
        """
        super().__init__()
        self.leadtimes_to_evaluate = leadtimes_to_evaluate
        self.add_state("pred_sie", default=torch.tensor(0.0), dist_reduce_fx="sum")
        self.add_state("true_sie", default=torch.tensor(0.0), dist_reduce_fx="sum")

    def update(
        self, preds: torch.Tensor, target: torch.Tensor, sample_weight: torch.Tensor
    ):
        # preds and target are shape (b, h, w, t)
        preds = (preds > 0.15).long()
        target = (target > 0.15).long()
        self.pred_sie += preds[:, :, :, self.leadtimes_to_evaluate].sum()
        self.true_sie += target[:, :, :, self.leadtimes_to_evaluate].sum()

    def compute(self):
        return (self.pred_sie - self.true_sie) * 25**2  # each pixel is 25x25 km

Define custom loss functions

In [ ]:
class WeightedBCEWithLogitsLoss(nn.BCEWithLogitsLoss):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def forward(self, inputs, targets, sample_weights):
        """
        Weighted BCEWithLogitsLoss loss.

        Compute BCEWithLogitsLoss loss weighted by masking.

        Using BCEWithLogitsLoss instead of BCELoss, as pytorch docs mentions it is
        more numerically stable.
        https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html

        """
        # Computing using nn.BCEWithLogitsLoss base class. This class must be instantiated via:
        # >>> criterion = WeightedBCEWithLogitsLoss(reduction='none')
        loss = super().forward(
            (inputs.movedim(-2, 1)), (targets.movedim(-1, 1))
        ) * sample_weights.movedim(-1, 1)

        return loss.mean()


class WeightedL1Loss(nn.L1Loss):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def forward(self, inputs, targets, sample_weights):
        """
        Weighted L1 loss.

        Compute L1 loss weighted by masking.

        """
        y_hat = torch.sigmoid(inputs)

        # Computing using nn.L1Loss class. This class must be instantiated via:
        # >>> criterion = WeightedL1Loss(reduction="none")
        loss = super().forward(
            (100 * y_hat.movedim(-2, 1)), (100 * targets.movedim(-1, 1))
        ) * sample_weights.movedim(-1, 1)

        # Computing here, in the derived class
        # loss = (
        #             torch.abs( ( y_hat.movedim(-2, 1) - targets.movedim(-1, 1) )*100 )
        #         )*sample_weights.movedim(-1, 1)

        return loss.mean()


class WeightedMSELoss(nn.MSELoss):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def forward(self, inputs, targets, sample_weights):
        """
        Weighted MSE loss.

        Compute MSE loss weighted by masking.

        """
        y_hat = torch.sigmoid(inputs)

        # Computing using nn.MSELoss base class. This class must be instantiated via:
        # criterion = nn.MSELoss(reduction="none")
        loss = super().forward(
            (100 * y_hat.movedim(-2, 1)), (100 * targets.movedim(-1, 1))
        ) * sample_weights.movedim(-1, 1)

        # Computing here, in the nn.Module derived class
        # loss = (
        #             ( ( y_hat.movedim(-2, 1) - targets.movedim(-1, 1) )*100 )**2
        #         )*sample_weights.movedim(-1, 1)
        return loss.mean()

A _LightningModule_ wrapper for UNet model.

In [ ]:
import lightning.pytorch as pl
from lightning.pytorch.utilities.types import TRAIN_DATALOADERS
from torchmetrics import MetricCollection


class LitUNet(pl.LightningModule):
    """
    A LightningModule wrapping the UNet implementation of IceNet.
    """

    def __init__(self, model: nn.Module, criterion: callable, learning_rate: float):
        """
        Construct a UNet LightningModule.
        Note that we keep hyperparameters separate from dataloaders to prevent data leakage at test time.
        :param model: PyTorch model
        :param criterion: PyTorch loss function for training instantiated with reduction="none"
        :param learning_rate: Float learning rate for our optimiser
        """
        super().__init__()
        self.model = model
        self.criterion = criterion
        self.learning_rate = learning_rate
        self.n_output_classes = (
            model.n_output_classes
        )  # this should be a property of the network

        metrics = {
            "val_accuracy": IceNetAccuracy(
                leadtimes_to_evaluate=list(range(self.model.n_forecast_days))
            ),
            "val_sieerror": SIEError(
                leadtimes_to_evaluate=list(range(self.model.n_forecast_days))
            ),
        }
        for i in range(self.model.n_forecast_days):
            metrics[f"val_accuracy_{i}"] = IceNetAccuracy(leadtimes_to_evaluate=[i])
            metrics[f"val_sieerror_{i}"] = SIEError(leadtimes_to_evaluate=[i])
        self.metrics = MetricCollection(metrics)

        test_metrics = {
            "test_accuracy": IceNetAccuracy(
                leadtimes_to_evaluate=list(range(self.model.n_forecast_days))
            ),
            "test_sieerror": SIEError(
                leadtimes_to_evaluate=list(range(self.model.n_forecast_days))
            ),
        }
        for i in range(self.model.n_forecast_days):
            test_metrics[f"test_accuracy_{i}"] = IceNetAccuracy(
                leadtimes_to_evaluate=[i]
            )
            test_metrics[f"test_sieerror_{i}"] = SIEError(leadtimes_to_evaluate=[i])
        self.test_metrics = MetricCollection(test_metrics)

        # Save input parameters to __init__ (hyperparams) when checkpointing.
        # self.save_hyperparameters(ignore=["model", "criterion"])
        self.save_hyperparameters()

    def forward(self, x):
        """
        Implement forward function.
        :param x: Inputs to model.
        :return: Outputs of model.
        """
        return self.model(x)

    def training_step(self, batch):
        """
        Perform a pass through a batch of training data.
        Apply pixel-weighted loss by manually reducing.
        See e.g. https://discuss.pytorch.org/t/unet-pixel-wise-weighted-loss-function/46689/5.
        :param batch: Batch of input, output, weight triplets
        :param batch_idx: Index of batch
        :return: Loss from this batch of data for use in backprop
        """
        x, y, sample_weight = batch
        outputs = self.model(x)
        # y_hat = torch.sigmoid(outputs)

        # loss = loss_func(outputs, y, sample_weight)
        loss = self.criterion(outputs, y, sample_weight)

        self.log(
            "train_loss",
            loss,
            on_step=True,
            on_epoch=True,
            prog_bar=True,
            sync_dist=True,
        )
        return {"loss": loss}

    def validation_step(self, batch):
        # x: (b, h, w, channels), y: (b, h, w, n_forecast_days, classes), sample_weight: (b, h, w, n_forecast_days, classes)
        x, y, sample_weight = batch
        # y_hat: (b, h, w, classes, n_forecast_days)
        outputs = self.model(x)
        y_hat = torch.sigmoid(outputs)

        # loss = loss_func(outputs, y, sample_weight)
        loss = self.criterion(outputs, y, sample_weight)

        self.log(
            "val_loss",
            loss,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            sync_dist=True,
        )  # epoch-level loss

        self.metrics.update(
            y_hat.squeeze(dim=-2), y.squeeze(dim=-1), sample_weight.squeeze(dim=-1)
        )

        return {"val_loss", loss}

    def on_validation_epoch_end(self):
        self.log_dict(
            self.metrics.compute(),
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            sync_dist=True,
        )  # epoch-level metrics
        self.metrics.reset()

    def test_step(self, batch):
        x, y, sample_weight = batch
        outputs = self.model(x)
        y_hat = torch.sigmoid(outputs)

        # loss = loss_func(outputs, y, sample_weight)
        loss = self.criterion(outputs, y, sample_weight)

        self.log(
            "test_loss",
            loss,
            on_step=False,
            on_epoch=True,
            prog_bar=True,
            sync_dist=True,
        )  # epoch-level loss

        self.test_metrics.update(
            y_hat.squeeze(dim=-2), y.squeeze(dim=-1), sample_weight.squeeze(dim=-1)
        )

        return loss

    def on_test_epoch_end(self):
        self.log_dict(
            self.test_metrics.compute(), on_step=False, on_epoch=True, sync_dist=True
        )  # epoch-level metrics
        self.test_metrics.reset()

    def predict_step(self, batch):
        """
        :param batch: Batch of input, output, weight triplets
        :param batch_idx: Index of batch
        :return: Predictions for given input.
        """
        x, y, sample_weight = batch
        y_hat = torch.sigmoid(self.model(x))

        return y_hat

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return {"optimizer": optimizer}

    # def train_dataloader(self) -> TRAIN_DATALOADERS:
    #     return super().train_dataloader()

Function for training UNet model using PyTorch Lightning.

In [ ]:
from lightning.pytorch.callbacks import ModelCheckpoint


def train_icenet(
    configuration_path,
    learning_rate,
    max_epochs,
    batch_size,
    n_workers,
    filter_size,
    n_filters_factor,
    seed,
):
    """
    Train IceNet using the arguments specified in the `args` namespace.
    :param args: Namespace of configuration parameters
    """
    # init
    pl.seed_everything(seed)

    # configure datasets and dataloaders
    train_dataset = IceNetDataSetPyTorch(configuration_path, mode="train")
    val_dataset = IceNetDataSetPyTorch(configuration_path, mode="val")
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=n_workers,
        persistent_workers=persistent_workers,
        shuffle=False,
    )
    val_dataloader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        num_workers=n_workers,
        persistent_workers=persistent_workers,
        shuffle=False,
    )

    # construct unet
    model = UNet(
        input_channels=train_dataset._num_channels,
        filter_size=filter_size,
        n_filters_factor=n_filters_factor,
        n_forecast_days=train_dataset._n_forecast_days,
    )

    # criterion = WeightedBCEWithLogitsLoss(reduction="none")
    # criterion = WeightedL1Loss(reduction="none")
    criterion = WeightedMSELoss(reduction="none")

    # configure PyTorch Lightning module
    lit_module = LitUNet(model=model, criterion=criterion, learning_rate=learning_rate)

    # set up trainer configuration
    trainer = pl.Trainer(
        accelerator="auto",
        devices=1,
        log_every_n_steps=5,
        max_epochs=max_epochs,
        num_sanity_val_steps=1,
        fast_dev_run=False,  # Runs single batch through train and validation
        #    when running trainer.test()
        # Note: Cannot use with automatic best checkpointing
    )
    checkpoint_callback = ModelCheckpoint(monitor="val_accuracy", mode="max")
    trainer.callbacks.append(checkpoint_callback)

    # train model
    print(
        f"Training {len(train_dataset)} examples / {len(train_dataloader)} batches (batch size {batch_size})."
    )
    print(
        f"Validating {len(val_dataset)} examples / {len(val_dataloader)} batches (batch size {batch_size})."
    )
    trainer.fit(lit_module, train_dataloader, val_dataloader)

    return model, trainer, checkpoint_callback

Conduct actual training run.

In [ ]:
seed = 45
model, trainer, checkpoint_callback = train_icenet(
    configuration_path=dataset_config,
    learning_rate=1e-4,
    max_epochs=10,
    batch_size=batch_size,
    n_workers=num_workers,
    filter_size=3,
    n_filters_factor=0.4,
    seed=seed,
)

## 4. Prediction

Predicts using the best checkpoint from the training.

In [ ]:
checkpoint_callback.best_k_models

In [ ]:
best_checkpoint = checkpoint_callback.best_model_path
best_checkpoint

In [ ]:
# Load the best result from the checkpoint
best_model = LitUNet.load_from_checkpoint(best_checkpoint)

# disable randomness, dropout, etc...
best_model.eval()

In [ ]:
test_dataset = IceNetDataSetPyTorch(configuration_path=dataset_config, mode="test")
test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    persistent_workers=persistent_workers,
    shuffle=False,
)

# automatically load the best weights (if best_model isn't added)
trainer.test(dataloaders=test_dataloader)

In [ ]:
logging.info("Generating predictions")

predictions = trainer.predict(best_model, dataloaders=test_dataloader)

In [ ]:
for worker, prediction in enumerate(predictions):
    print(f"Worker: {worker} | Prediction: {prediction.shape}")

## 5. Outputs and Plotting

Create prediction output directory

In [ ]:
# dataset = "pytorch_notebook"
network_name = "api_pytorch_dataset"
output_name = "example_pytorch_forecast"
output_folder = os.path.join(
    ".", "results", "predict", output_name, "{}.{}".format(network_name, seed)
)
os.makedirs(output_folder, exist_ok=output_folder)

Convert and output predictions to numpy files

In [ ]:
idx = 0
for workers, prediction in enumerate(predictions):
    for batch in range(prediction.shape[0]):
        date = pd.Timestamp(test_dataset.dates[idx].replace("_", "-"))
        output_path = os.path.join(output_folder, date.strftime("%Y_%m_%d.npy"))
        forecast = prediction[batch, :, :, :, :].movedim(-2, 0)
        forecast_np = forecast.detach().cpu().numpy()
        np.save(output_path, forecast_np)
        idx += 1

Create a csv file with all the test dates we have predicted for, and to use in generating the final netCDF output using `icenet_output`.

In [ ]:
!for day in $(seq -w 1 31); do printf "2019-12-${day}\n"; done | tee predict_dates.csv

In [ ]:
import sys

!{sys.executable.replace('python', 'icenet_output')}  -m -o results/predict example_pytorch_forecast notebook_api_pytorch_data predict_dates.csv

Plotting the forecast

In [ ]:
import xarray as xr
import datetime as dt
from IPython.display import HTML

In [ ]:
from icenet.plotting.video import xarray_to_video as xvid
from icenet.data.sic.mask import Masks

ds = xr.open_dataset("results/predict/example_pytorch_forecast.nc")
land_mask = Masks(south=True, north=False).get_land_mask()
ds.info()

Animate result

In [ ]:
forecast_date = ds.time.values[0]
fc = ds.sic_mean.isel(time=0).drop_vars("time").rename(dict(leadtime="time"))
fc["time"] = [
    pd.to_datetime(forecast_date) + dt.timedelta(days=int(e)) for e in fc.time.values
]

# anim = xvid(fc, 15, figsize=4, mask=land_mask)
anim = xvid(fc, 15, figsize=(4, 4), mask=land_mask, north=False, south=True)
HTML(anim.to_jshtml())

Check min/max of predicted SIC fraction

In [ ]:
print(forecast_np[:, :, :, 0].shape)
fmin, fmax = np.min(forecast_np[:, :, :, 0]), np.max(forecast_np[:, :, :, 0])
print(f"First forecast day min: {fmin:.4f}, max: {fmax:.4f}")

#### Load original input dataset

This is the original input dataset (pre-normalisation) for comparison.

In [ ]:
# Load original input dataset (domain not normalised)
xr.plot.contourf(
    xr.open_dataset("data/osisaf/south/siconca/2020.nc").isel(time=92).ice_conc,
    levels=50,
)

## Version
- IceNet Codebase: v0.2.8